# START YOUR CODE HERE IN THIS NOTEBOOK, SIR/MS. :)

In [8]:
import os

modelList = [] 

for x in os.listdir("model"): 
    modelList += [x]
    
print(modelList)

['archive', 'PDAN_TSU_RGB.zip']


In [11]:
import zipfile, torch
import ipywidgets as widgets
from ipywidgets import interact, Dropdown 
from IPython.display import display

models = widgets.Dropdown(options = modelList)
load_button = widgets.Button(description="Load")
output = widgets.Output()
selected_model = ""

In [12]:
def onClickLoadModel(b):
    selected_model = models.value
    if (selected_model.split('.')[-1] == "zip"):
        with zipfile.ZipFile("model/"+selected_model,"r") as zip_file:
            zip_file.extractall("model/")
    pretrained_model = torch.load("model/archive/data.pkl") 

In [13]:
load_button.on_click(onClickLoadModel)

In [14]:
widgets.HBox([models,load_button])

ModuleNotFoundError: No module named 'models'

In [15]:
!python --version

Python 3.8.13
